In [68]:
# Necessary Libraries.
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import yfinance as yf
import os
from dotenv import load_dotenv
import datetime

In [69]:
import pandas as pd

# Load the data
gild_data = pd.read_csv('../data/raw/GILD_stock_data.csv')
all_data = pd.read_csv('../data/raw/GILD_2023-01-01_to_2023-08-15.csv')

# Convert 'Date' column in gild_data to datetime and set it as the index
gild_data['Date'] = pd.to_datetime(gild_data['Date'])
gild_data.set_index('Date', inplace=True)

# Convert 'transactionDate' column in all_data to datetime
all_data['transactionDate'] = pd.to_datetime(all_data['transactionDate'])

# Process all_data to determine the type of transaction
all_data['insiderPortfolioChange'] = all_data['change'] / (all_data['share'] - all_data['change'])
conditions = [
    (all_data['change'] >= 0) & (all_data['transactionPrice'] > 0),
    (all_data['change'] <= 0) & (all_data['transactionPrice'] > 0),
    (all_data['transactionPrice'] == 0)
]
values = ['Buy', 'Sale', 'Gift']
all_data['buyOrSale'] = pd.np.select(conditions, values)
gild_data.index = pd.to_datetime(gild_data.index, utc=True).tz_convert(None)
gild_data.index = gild_data.index.normalize()
gild_data.tail(5)


/var/folders/f8/g9fbmmyn3757__nzcw0m__dm0000gn/T/ipykernel_5656/201137906.py:22: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  all_data['buyOrSale'] = pd.np.select(conditions, values)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,EPS Estimate,Reported EPS,Surprise(%)
Date,,,,,,,,,,
2023-08-09,80.570000,81.430000,80.029999,80.169998,4551300,0.0,0.0,NaN,NaN,NaN
2023-08-10,80.379997,81.089996,79.949997,80.000000,3776200,0.0,0.0,NaN,NaN,NaN
2023-08-11,80.000000,80.129997,79.459999,80.000000,3383600,0.0,0.0,NaN,NaN,NaN
2023-08-14,80.059998,80.419998,79.519997,79.849998,3682000,0.0,0.0,NaN,NaN,NaN
2023-08-15,79.629997,80.209999,78.959999,79.639999,3798600,0.0,0.0,NaN,NaN,NaN


In [70]:
# Number of records
num_records = all_data.shape[0]
print(f"Number of Records: {num_records}")

# Number of duplicates
num_duplicates = all_data.duplicated().sum()
print(f"Number of Duplicates: {num_duplicates}")

# Number of null values for each column
null_values = all_data.isnull().sum()
print("\nNumber of Null Values for Each Column:")
print(null_values)

# Summary statistics
print("\nSummary Statistics:")
print(all_data.describe())

# More detailed information:
print("\nDataFrame Info:")
print(gild_data.info())

Number of Records: 116
Number of Duplicates: 0

Number of Null Values for Each Column:
change                    0
filingDate                0
id                        0
name                      0
share                     0
source                    0
symbol                    0
transactionCode           0
transactionDate           0
transactionPrice          0
insiderPortfolioChange    0
buyOrSale                 0
dtype: int64

Summary Statistics:
              change          share  transactionPrice  insiderPortfolioChange
count     116.000000     116.000000        116.000000              116.000000
mean     8069.310345   75214.982759         25.412271                     inf
std     29696.499736   92229.527897         36.852781                     NaN
min    -59078.000000       0.000000          0.000000               -1.000000
25%     -3025.000000   22182.750000          0.000000               -0.055535
50%      1692.500000   45714.500000          0.000000                0.0282

In [71]:

# Aggregate data based on transaction date
aggregated_data = all_data.groupby('transactionDate').agg({
    'change': 'sum',
    'transactionPrice': 'mean',
    'buyOrSale': lambda x: x.value_counts().to_dict()
}).reset_index()

# Split the buyOrSale dictionary column into separate columns
aggregated_data['Number of Buys'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Buy', 0))
aggregated_data['Number of Sells'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Sale', 0))
aggregated_data['Number of Gifts'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Gift', 0))
aggregated_data.drop('buyOrSale', axis=1, inplace=True)
# Convert 'transactionDate' column in all_data to datetime
aggregated_data['transactionDate'] = pd.to_datetime(aggregated_data['transactionDate'])
aggregated_data['transactionDate'] = aggregated_data['transactionDate'].dt.normalize()
aggregated_data.tail()


,transactionDate,change,transactionPrice,Number of Buys,Number of Sells,Number of Gifts
12,2023-06-29,31,0.000000,0,0,1
13,2023-07-20,-5000,80.000000,0,1,0
14,2023-07-24,-3174,25.886667,0,1,2
15,2023-07-25,-4174,15.446000,0,1,4
16,2023-08-10,-1306,26.666667,0,1,2


In [72]:
# Merge the dataframes
final_df = pd.merge(gild_data, aggregated_data, left_index=True, right_on='transactionDate', how='left')
final_df.set_index('transactionDate', inplace=True)


# Display the final DataFrame
print(final_df.tail(20))


                      Open       High        Low      Close    Volume  \
transactionDate                                                         
2023-07-19       78.099998  79.599998  78.099998  78.959999   5351600   
2023-07-20       79.559998  80.779999  79.540001  79.940002   6046000   
2023-07-21       80.410004  81.129997  80.000000  80.639999  14982100   
2023-07-24       79.820000  80.180000  76.250000  77.660004  11333500   
2023-07-25       77.019997  78.150002  76.860001  77.230003   5298400   
2023-07-26       76.769997  77.379997  76.610001  77.190002   4049700   
2023-07-27       77.330002  78.279999  76.400002  76.510002   6258600   
2023-07-28       76.419998  77.320000  76.339996  76.860001   5026500   
2023-07-31       76.669998  76.889999  75.849998  76.139999   6059500   
2023-08-01       76.089996  76.379997  75.099998  75.680000   4025300   
2023-08-02       76.000000  76.320000  75.349998  75.680000   5325400   
2023-08-03       75.720001  75.820000  75.169998  7